In [37]:
%load_ext kedro.ipython

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


In [38]:
%reload_ext kedro.ipython

[05/16/24 21:18:27] INFO     Registered line magic '%reload_kedro'                                   ]8;id=111395;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=405708;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#51\51]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=251409;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=687515;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#53\53]8;;\

                    INFO     Resolved project path as: /Users/gavinlou/Developer/gasai.             ]8;id=138101;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=382456;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#164\164]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

                    INFO     Kedro project gasAI                                                    ]8;id=854459;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=663751;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#134\134]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=796267;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=741560;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#135\135]8;;\
                             'pipelines'                                                                           

[05/16/24 21:18:28] INFO     Registered line magic 'run_viz'                                        ]8;id=272837;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=768096;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/ipython/__init__.py#141\141]8;;\

In [39]:
df = catalog.load("s3_conc_aligned_df")

                    INFO     Loading data from s3_conc_aligned_df (ParquetDataset)...           ]8;id=858400;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=90160;file:///Users/gavinlou/.pyenv/versions/3.10.13/envs/kedro/lib/python3.10/site-packages/kedro/io/data_catalog.py#483\483]8;;\

In [40]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler

class GroupDataset(Dataset):
    def __init__(self, X, y, groups):
        self.X = X.unsqueeze(2)
        self.y = y.unsqueeze(1)
        self.groups = groups
        self.group_to_indices = self._group_indices()

    def _group_indices(self):
        group_to_indices = {}
        for idx, group in enumerate(self.groups):
            if group not in group_to_indices:
                group_to_indices[group] = []
            group_to_indices[group].append(idx)
        return group_to_indices

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.groups[idx]

class ExperimentSampler:
    def __init__(self, group_to_indices, batch_size):
        self.group_to_indices = group_to_indices
        self.batch_size = batch_size
        self.group_order = list(group_to_indices.keys())
        np.random.shuffle(self.group_order)  # Shuffle the order of groups

    def __iter__(self):
        for group in self.group_order:
            indices = self.group_to_indices[group]
            for i in range(0, len(indices), self.batch_size):
                yield indices[i:i + self.batch_size]

    def __len__(self):
        total_batches = 0
        for indices in self.group_to_indices.values():
            total_batches += (len(indices) + self.batch_size - 1) // self.batch_size
        return total_batches

# Define the features and target
features = ['timestamp_bin', 'A1_Resistance']
target = 'resistance_ratio'
X_tensor = df[features]
y_tensor = df[target]
groups = df['exp_no']
batch_size = 32

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_tensor.values, dtype=torch.float32)
y_tensor = torch.tensor(y_tensor.values, dtype=torch.float32)
# Convert groups to NumPy array
groups = np.array(groups)

# Create dataset
dataset = GroupDataset(X_tensor, y_tensor, groups)

# 1. Split the data into training and testing sets using GroupShuffleSplit
gss = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)
train_idx, test_idx = next(gss.split(X_tensor, y_tensor, groups=groups))

# 2. Fit the scaler on the training data only
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_tensor[train_idx].numpy())  # Convert to NumPy if necessary and scale

# 3. Transform the test data using the same scaler
X_test_scaled = scaler.transform(X_tensor[test_idx].numpy())

# Convert scaled data back to tensors
X_train_scaled_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_scaled_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

# 4. Create datasets
train_dataset = GroupDataset(X_train_scaled_tensor, y_tensor[train_idx], groups[train_idx])
test_dataset = GroupDataset(X_test_scaled_tensor, y_tensor[test_idx], groups[test_idx])

# 5. Create experiment samplers
train_sampler = ExperimentSampler(train_dataset.group_to_indices, batch_size)
test_sampler = ExperimentSampler(test_dataset.group_to_indices, batch_size)

# 6. Create data loaders
train_loader = DataLoader(train_dataset, batch_sampler=train_sampler)
test_loader = DataLoader(test_dataset, batch_sampler=test_sampler)


In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("mps" if torch.cuda.is_available() else "cpu")

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        out, _ = self.lstm(x, (h0,c0))  
        out = out[:, -1, :]
        out = self.fc(out)
        return out
    
    

In [42]:
from torch.optim import Adam
from math import sqrt

input_size = X_tensor.shape[1]
hidden_size = 64
num_layers = 2
num_classes = 1

device = torch.device("mps" if torch.cuda.is_available() else "cpu")

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)

def calculate_rmse(outputs, labels):
    mse = torch.mean((outputs - labels) ** 2)  # Calculate MSE as a tensor
    return torch.sqrt(mse)  # Return RMSE as a tensor



def train_model(model, train_loader, val_loader, n_epochs):
    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels, _ in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            # print(inputs.shape)
            inputs = inputs.transpose(1, 2)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation
        model.eval()

        # During validation
        val_rmse = 0.0
        total_samples = 0
        with torch.no_grad():
            for inputs, labels, _ in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                inputs = inputs.transpose(1, 2)
                outputs = model(inputs)
                batch_rmse = calculate_rmse(outputs, labels) * inputs.size(0)
                val_rmse += batch_rmse.item()  # Properly use .item() to convert tensor to float
                total_samples += inputs.size(0)

        val_rmse /= total_samples
        print(f'Epoch {epoch+1} Train Loss: {epoch_loss:.4f} Val RMSE: {val_rmse:.4f}')

# Training the model
train_model(model, train_loader, test_loader, n_epochs=30)


Epoch 1 Train Loss: 0.0433 Val RMSE: 0.1540
Epoch 2 Train Loss: 0.0111 Val RMSE: 0.1014
Epoch 3 Train Loss: 0.0083 Val RMSE: 0.0933
Epoch 4 Train Loss: 0.0077 Val RMSE: 0.0813
Epoch 5 Train Loss: 0.0072 Val RMSE: 0.0723
Epoch 6 Train Loss: 0.0067 Val RMSE: 0.0655
Epoch 7 Train Loss: 0.0060 Val RMSE: 0.0587
Epoch 8 Train Loss: 0.0052 Val RMSE: 0.0541
Epoch 9 Train Loss: 0.0047 Val RMSE: 0.0546
Epoch 10 Train Loss: 0.0045 Val RMSE: 0.0569
Epoch 11 Train Loss: 0.0044 Val RMSE: 0.0583
Epoch 12 Train Loss: 0.0043 Val RMSE: 0.0591
Epoch 13 Train Loss: 0.0043 Val RMSE: 0.0594
Epoch 14 Train Loss: 0.0042 Val RMSE: 0.0591
Epoch 15 Train Loss: 0.0041 Val RMSE: 0.0587
Epoch 16 Train Loss: 0.0040 Val RMSE: 0.0577
Epoch 17 Train Loss: 0.0040 Val RMSE: 0.0566
Epoch 18 Train Loss: 0.0039 Val RMSE: 0.0555
Epoch 19 Train Loss: 0.0039 Val RMSE: 0.0545
Epoch 20 Train Loss: 0.0038 Val RMSE: 0.0536
Epoch 21 Train Loss: 0.0038 Val RMSE: 0.0529
Epoch 22 Train Loss: 0.0037 Val RMSE: 0.0528
Epoch 23 Train Loss

In [43]:
# i want to examine the content of the loader to see if timestamps are being passed correctly

for inputs, label, group in train_loader:
    print(inputs)
    break


tensor([[[-1.7110],
         [-0.7558]],

        [[-1.7090],
         [-0.7601]],

        [[-1.7070],
         [-0.7616]],

        [[-1.7051],
         [-0.7656]],

        [[-1.7031],
         [-0.7698]],

        [[-1.7011],
         [-0.7743]],

        [[-1.6991],
         [-0.7839]],

        [[-1.6971],
         [-0.7884]],

        [[-1.6952],
         [-0.8000]],

        [[-1.6932],
         [-0.8073]],

        [[-1.6912],
         [-0.8161]],

        [[-1.6892],
         [-0.8248]],

        [[-1.6873],
         [-0.8334]],

        [[-1.6853],
         [-0.8414]],

        [[-1.6833],
         [-0.8491]],

        [[-1.6813],
         [-0.8598]],

        [[-1.6794],
         [-0.8671]],

        [[-1.6774],
         [-0.8740]],

        [[-1.6754],
         [-0.8812]],

        [[-1.6734],
         [-0.8870]],

        [[-1.6714],
         [-0.8928]],

        [[-1.6695],
         [-0.9012]],

        [[-1.6675],
         [-0.9058]],

        [[-1.6655],
         [-0.9

In [44]:
print(X_tensor.shape)

torch.Size([469995, 2])
